<a href="https://colab.research.google.com/github/chamchiiii/chamchiiii.github.io/blob/main/Week2_Mission6_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B_%EC%9A%94%EC%95%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import re
from collections import Counter

file_path = '/content/drive/My Drive/Labeled_Data_Week2_Mission_SkinType_Ingredient_AI_KR - Sheet1.csv'
df = pd.read_csv(file_path)

df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,product_id,brand,product_name,ingredient_raw,skin_type_label,skin_concern_label
0,1,Paula's Choice,CellularYouth™ Age-Disrupting Longevity Serum,"water, propanediol, glycerin, triheptanoin, me...","건성, 지성, 복합성, 중성","주름/노화, 피부결 개선"
1,2,Paula's Choice,25% Vitamin C + Glutathione Clinical Serum,"water, tetrahexyldecyl ascorbate, dimethicone,...",NaN,잡티/색소침착
2,3,Paula's Choice,RESIST Triple Active Total Repair Serum,"water, isopropyl myristate, caprylic/capric tr...",NaN,"주름/노화, 잡티/색소침착, 건조/보습 부족, 장벽 손상"
3,4,Paula's Choice,7% Ectoin + Hyaluronic Acid Milky Hydrating Serum,"water, ectoin, isopentyldiol, triethylhexanoin...",건성,"장벽 손상, 민감/자극, 건조/보습 부족"
4,5,Paula's Choice,CLINICAL Niacinamide 20% Treatment,"water, niacinamide, pentylene glycol, butylene...",NaN,"피부결 개선, 주름/노화, 자외선 노출, 잡티/색소침착"


In [8]:
total_products=len(df)
total_products

100

In [9]:
skin_type_counter = Counter()

for types in df['skin_type_label'].dropna():
    for t in str(types).split(","):
        skin_type_counter[t.strip()] += 1

skin_type_distribution = pd.DataFrame(skin_type_counter.items(),columns=["Skin Type", "Product Count"]).sort_values(by="Product Count", ascending=False)

skin_type_distribution


,Skin Type,Product Count
0,건성,11
4,민감성,8
1,지성,6
2,복합성,6
3,중성,3


In [12]:
skin_concern_counter = Counter()

for types in df['skin_concern_label'].dropna():
    for t in str(types).split(","):
        skin_concern_counter[t.strip()] += 1

skin_concern_distribution = pd.DataFrame(skin_concern_counter.items(),columns=["Skin Concern", "Product Count"]).sort_values(by="Product Count", ascending=False)

skin_concern_distribution

,Skin Concern,Product Count
0,주름/노화,56
2,잡티/색소침착,48
3,건조/보습 부족,48
1,피부결 개선,32
5,민감/자극,23
4,장벽 손상,12
7,여드름/트러블,12
6,자외선 노출,4


In [10]:
df['ingredient_count'] = df['ingredient_raw'].apply(
    lambda x: len(str(x).split(",")) if pd.notna(x) else 0
)

average_ingredients = df['ingredient_count'].mean()
float(average_ingredients)


29.22

In [24]:
# 1. 성분 빈도 계산
ingredient_counter = Counter()

for ing_list in df['ingredient_raw'].dropna():
    for ing in re.split(r'(?<!\d),(?!\d)', str(ing_list)):
        ingredient_counter[ing.strip()] += 1

# 2. Counter → DataFrame
ingredient_freq = pd.DataFrame(
    ingredient_counter.items(),
    columns=["Ingredient", "Frequency"]
)

# 3. 정규화 (소문자 + 동의어)
synonym_map = {'aqua': 'water', 'water': 'water'}

def normalize_ingredient(name):
    name = name.lower().strip()
    return synonym_map.get(name, name)

ingredient_freq['ingredient_norm'] = ingredient_freq['Ingredient'].apply(normalize_ingredient)

# 4. 같은 성분 합산
merged_freq = (
    ingredient_freq
    .groupby('ingredient_norm', as_index=False)['Frequency']
    .sum()
    .sort_values(by='Frequency', ascending=False)
)

merged_freq.head(10)

,ingredient_norm,Frequency
734,water,80
263,glycerin,70
231,ethylhexylglycerin,63
94,butylene glycol,54
736,xanthan gum,53
558,propanediol,51
466,phenoxyethanol,49
629,sodium hyaluronate,46
698,tocopherol,44
0,"1,2-hexanediol",43


In [20]:
#전체 고유 성분 수
print(len(merged_freq))


745


In [32]:
ingredient_vocab = merged_freq['ingredient_norm'].tolist()

ingredient_to_index = {ingredient: idx for idx, ingredient in enumerate(ingredient_vocab)}

index_to_ingredient = {idx: ingredient for ingredient, idx in ingredient_to_index.items()}





NameError: name 'product_ingredients' is not defined